In [6]:
import os
from decouple import config
import second_level_functions as slf
import numpy as np
import pandas as pd

In [2]:
task = 'eft'
threshold_pval = -np.log10(0.05)

In [11]:
base_dir = config(task)
path = os.path.join(base_dir, '2ndlevel', 'mixed_model')

images = slf.get_images(path)
contrasts = slf.contrast_imgs(images['t_stat'], images['pvals'])

In [25]:
def participant_data(base_dir: str) -> pd.DataFrame:

    '''
    Function to read in and build df of participants information.

    Parameters
    ----------
    base_dir: str
        directory where 1stlevel_location.csv is located.

    Returns
    -------
    long_df: pd.DataFrame
        DataFrame in long form of subject, time and id values
    '''
    participant_scans = pd.read_csv(f"{base_dir}/1stlevel_location.csv")
    participant_scans['sub'] = participant_scans.index
    long_df = pd.melt(participant_scans, id_vars=['sub'], 
                      var_name='time_point', 
                      value_vars=['t1', 't2',], 
                      value_name='scans').sort_values(by=['sub'], ascending=True).reset_index(drop=True)
    long_df = long_df.drop(long_df[long_df['sub'] == 75].index)
    long_df['scans'] = long_df['scans'].str.replace(r'/data/project/BEACONB/task_fmri/.*?/1stlevel/T./', '', regex=True)
    long_df['scans'] = long_df['scans'].str.replace(r'/con.*', '', regex=True)
    long_df['group'] = long_df['scans'].apply(lambda participants: 'HC' if 'sub-G1' in participants or 'sub-B1' in participants else 'AN')

    return long_df

In [4]:
fmri_img_mask = slf.build_mask(contrasts['interaction']['pvals'], threshold_pval)

In [5]:
parameter_estimates = slf.extract_parameter_estimates(path, fmri_img_mask)

In [28]:
parameter_estimates_df = pd.concat((long_df, pd.DataFrame(parameter_estimates)), axis=1)